In [ ]:
import numpy
import pandas

import seaborn as sns

from matplotlib import pyplot


sns.set_theme(font="IPAexGothic")
pyplot.rcParams["figure.figsize"] = (20, 10)


In [ ]:
url="https://www.mof.go.jp/jgbs/reference/interest_rate/data/jgbcm_all.csv"
_df = pandas.read_csv(url, header=1, encoding="shift-jis")
_df.tail()


In [ ]:
def convert_wareki_to_seireki(wareki_date):
    """
    和暦を西暦に変換する関数。
    """
    era = wareki_date[0]
    year, month, day = map(int, wareki_date[1:].split('.'))

    if era == "S":  # 昭和
        seireki_year = 1925 + year
    elif era == "H":  # 平成
        seireki_year = 1988 + year
    elif era == "R":  # 令和
        seireki_year = 2018 + year
    else:
        raise ValueError(f"Unknown era: {era}")

    return f"{seireki_year}-{month:02}-{day:02}"


In [ ]:
_df["ds"] = pandas.to_datetime(_df["基準日"].apply(convert_wareki_to_seireki))
_df[["基準日", "ds"]]

In [ ]:
_df.replace('-', float('nan'), inplace=True)


In [ ]:
_df.head()

In [ ]:
rate_cols = [col for col in _df.columns if col not in ["基準日", "ds"]]
rate_cols = ["1年", "5年", "10年", "20年"]
rate_cols

In [ ]:
_df[rate_cols] = _df[rate_cols].astype(float)
_df.head(3)

In [ ]:
pyplot.figure(figsize=(15, 7))
for col in rate_cols:
    pyplot.plot(_df["ds"], _df[col], label=col)

pyplot.title("日本国債金利の変動")
pyplot.xlabel("ds")
pyplot.ylabel("金利 (%)")
pyplot.legend()
pyplot.grid(True)
pyplot.tight_layout()
pyplot.show()


In [ ]:
import optuna

from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics


In [ ]:
def build_prophet_model(**params) -> Prophet:
    model = Prophet(**params, yearly_seasonality=4, changepoint_range=1.0)
    model.add_seasonality(name='triennial', period=365.25*3, fourier_order=1)
    model.add_seasonality(name='kitchen', period=365.25/12*40, fourier_order=1)
    model.add_seasonality(name='quinquennial', period=365.25*5, fourier_order=1)
    model.add_seasonality(name='decennial_09', period=365.25*9, fourier_order=1)
    model.add_seasonality(name='decennial_10', period=365.25*10, fourier_order=1)
    return model


In [ ]:
class Evaluator(object):
   def __init__(self, df: pandas.DataFrame, n_horizon: int=31*12) -> None:
      self.df: pandas.DataFrame = df
      self.n_horizon: int = n_horizon

   def objective_value(self, trial: optuna.Trial):
      params = {
               "growth" : 
                  trial.suggest_categorical("growth", ["linear", "logistic", "flat"]),
               "changepoint_prior_scale" : 
                  trial.suggest_float("changepoint_prior_scale", 0.001, 10),
               "seasonality_prior_scale" : 
                  trial.suggest_float("seasonality_prior_scale", 0.01, 10),
               "seasonality_mode" : 
                  trial.suggest_categorical("seasonality_mode", ["additive", "multiplicative"])
               }

      model: Prophet = build_prophet_model(**params)
      model.fit(self.df)
      __df_cv, df_pm = self.run_cross_validation(model=model, df=self.df, horizon_scaler=3)
      score = numpy.cumsum(df_pm['rmse'].values[::-1]).mean() + numpy.cumsum(df_pm['mae'].values).mean()

      return score

   def run_cross_validation(self, model: Prophet, df: pandas.DataFrame, horizon_scaler: int=2, freq="D"):
      n_horizon = self.n_horizon
      date_start = df.ds.max() - pandas.Timedelta(days=n_horizon * horizon_scaler)
      date_end = df.ds.max() - pandas.Timedelta(days=n_horizon)
      cutoffs = pandas.date_range(start=date_start, end=date_end, freq=freq)
      df_cv = cross_validation(model, cutoffs=cutoffs, horizon=f"{n_horizon} days", parallel="processes")
      df_pm = performance_metrics(df_cv)
      return df_cv, df_pm



In [ ]:
from dataclasses import dataclass


In [ ]:
@dataclass
class Limitter(object):
    base_df: pandas.DataFrame

    def __post_init__(self):
        base_df = self.base_df
        self.floor = base_df.y.min() - 3 * base_df.y.std()
        self.cap = base_df.y.max() + 3 * base_df.y.std()

    def setup_limit(self, df: pandas.DataFrame, inplace=False) -> pandas.DataFrame:
        _df = df
        if not inplace:
            _df = df.copy()
        _df["floor"] = self.floor
        _df["cap"] = self.cap
        return _df


In [ ]:
@dataclass
class BestEstimator(object):
    df: pandas.DataFrame
    model: Prophet
    evaluator: Evaluator
    study: optuna.Study
    df_cv: dict
    df_pm: dict



In [ ]:
n_years = 2
n_predicts = 12 * n_years

bests = {}
for rate_type in rate_cols:
    # setup
    base_df = _df.rename({"年月": "ds", rate_type: "y"}, axis=1)[["ds", "y"]]
    lmt = Limitter(base_df)
    base_df = lmt.setup_limit(base_df)
    evl = Evaluator(df=base_df, n_horizon=31*12)

    # optimize hyper params
    study: optuna.Study = optuna.create_study(direction="minimize")
    study.optimize(evl.objective_value, n_trials=100)

    # rerun cv
    model: Prophet = build_prophet_model(**study.best_params)
    model.fit(base_df)
    df_cv, df_pm = evl.run_cross_validation(model=model, df=base_df, horizon_scaler=3)

    # predict
    future = model.make_future_dataframe(periods=n_predicts, freq="MS")
    future = lmt.setup_limit(future)
    forecast = model.predict(future)

    # store estimator
    bst = BestEstimator(df=base_df, model=model, evaluator=evl, study=study, df_cv=df_cv, df_pm=df_pm)
    bests[rate_type] = bst

